In [ ]:
#| hide
from pymoq.all import *

%pip install pytest -q
import pytest

Note: you may need to restart the kernel to use updated packages.


# General structure

In [ ]:
from typing import Protocol

class IWeb(Protocol):
    "Interface for accessing internet resources"
    
    def get(self, url:str, page:int, verbose:bool=False) -> str:
        "Fetches the ressource at `url` and returns it in string representation"

In [ ]:
class RessourceFetcher:
    base_url: str = "https://some_base.com/"
    
    def __init__(self, web: IWeb):
        self._web = web
    
    def check_ressource(self, ressource_name: str, page:int, verbose:bool=False) -> bool:
        url = self.base_url + ressource_name
        ressource = self._web.get(url, page, verbose)
        
        return ressource is not None

The general structure of pymoqs workflows is:

## Setup


In [ ]:
#| hide
ArbitrarySignatureValidator = ""
ArbitraryReturnValueGenerator = ""

In [ ]:
mock = Mock(IWeb)

mock.get\
    .setup(ArbitrarySignatureValidator)\
    .returns(ArbitraryReturnValueGenerator)

In general, a `SignatureValidator` is a list of `ArgumentValidator` `s`.

An object following the `ArgumentValidator` protocol has, among other properties, a `is_valid` method that accepts a single argument and returns a bool, indicating if the given argument matches the expected conditions. Conditions might be a type check, a direct value check or something else entirely (e.g. "is it a string that starts with 'py'?).

To make construcing a suitable list of `ArgumentValidator` `s` more convenient, there are a bunch of shortcuts for passing values to the `setup` method. They coded in `argument_validator_from_argument`. As of 2023-03-18, the shortcuts are:

- If an object is passed that satisfies the `ArgumentValidator` Protocol, it is used without any alteration
- If a type is passed, an `ArgumentFunctionValidator` is constructed that matches against that type
- If a Callable is passed, an `ArgumentFunctionValidator` is constructed that passes the argument through to the callable
- In any other case, an `ArgumentFunctionValidator` is constructed that matches the argument against the passed value


In the example:

In [ ]:
mock = Mock(IWeb)
mock.get\
    .setup('https://some_base.com/ressource', int, False);

- The first argument constructs a `ArgumentValidator` that returns true iff the string `https://some_base.com/ressource` is passed (last case)
- The second argument constructs a `ArgumentValidator` that returns true iff the passed argument is of type `int` (second case)
- The third argument constructs a `ArgumentValidator` that returns true iff the passed argument hast the value `False` (last case)

The first argument could also be an arbitrary function evaluation like:


In [ ]:
mock.get\
    .setup(lambda arg: isinstance(arg, str) and arg.startswith('https'), int, False);

This now matches against any `arg` that is of type `string` and starts with the substring `https`.

## Return Action

If the a call on a mock satisfies one of the setups, the corresponding return action is invoked:

In [ ]:
mock.get\
    .setup(ArbitrarySignatureValidator)\
    .returns(ArbitraryReturnValueGenerator)

The `ArbitraryReturnValueGenerator` is an object that follows the `ReturnValueGenerator` protocol. Essentially thats any callable. `pymoq` passes the arguments that were used in the specific call to the `ReturnValueGenerator`, enabling the user to return values depending on the concrete arguments used in each call.

To make constructing a `ReturnValueGenerator` more convenient, one can pass a non-callable object. `pymoq` constructs a `ReturnValueGenerator` from this that takes in any number of arguments and always returns that one value.

E.g.

In [ ]:
mock.get\
    .setup('https://some_base.com/ressource', int, False)\
    .returns(True)

assert mock.get('https://some_base.com/ressource', 0, False)
assert mock.get('https://some_base.com/ressource', 1, False)

will always return `True` (if the signature matches the validator in the setup function).

In contrast,

In [ ]:
mock.get\
    .setup('https://some_base.com/ressource', int, False)\
    .returns(lambda self, url, page, verbose: page+1)

assert mock.get('https://some_base.com/ressource', 0, False) == 1
assert mock.get('https://some_base.com/ressource', 5, False) == 6

will return `page + 1`, making the return value dependent on the caller value.

### Return sequence

It's possible to setup a sequence of return values. For each invocation that matches the signature validator, the next value of the sequence is returned. If the sequence is empty, `None` is returned.

In [ ]:
mock = Mock(IWeb)
mock.get.setup('resource', int, bool).returns_sequence([1,2,3])

assert mock.get('resource', 1, True)==1
assert mock.get('resource', 2, False)==2
assert mock.get('resource', 3, True)==3

print(mock.get('ressource', 1, True))

None


### Return exceptions

A return action could also be the throwing of an exception:

In [ ]:
class WebException(Exception):
    """Exception that describes web-access errors"""


mock = Mock(IWeb)
fetcher = RessourceFetcher(mock)

# setup failing web call
mock.get.setup('https://some_base.com/unavailable_ressource', int, bool).throws(WebException())

# act and assert exception
with pytest.raises(WebException):
    fetcher.check_ressource('unavailable_ressource', 1, True)
    
# does not raise exception if call signature does not match
fetcher.check_ressource('available_ressource', 1, True);


## Verification